# Feature Selection using BPSO for SDP

installations

In [1]:
 pip install pyswarms

Note: you may need to restart the kernel to use updated packages.


import libraries 

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import os
import pyswarms as ps
from scipy.io import arff

%load_ext autoreload
%autoreload 2
%matplotlib inline

### The methods 

Dataset prepration

In [3]:
def data_prepration(file):
    data = arff.loadarff(file)
    X=[]
    y=[]
    for i in data[0]:

        X.append(list(i)[0:21])
        if(i[len(i)-1] == b'Y'):
            y.append(1)
        else:
            y.append(0)
        
    X=np.array(X)
    y=np.array(y)
    return X,y

plot data per feature (each class with different color)

In [4]:
def plot_features(features,labels):
    df = pd.DataFrame(features)
    df['labels'] = pd.Series(labels)

    sns.pairplot(df, hue='labels' , diag_kind='hist')

fitness function for each particle

In [12]:
def f_per_particle(m):
    if(clas=='NB'):
        classifier = GaussianNB()
    elif(clas=='SVM linear'):
        classifier = svm.SVC(kernel='linear')
    elif(clas=='KNN'):
        classifier = KNeighborsClassifier(n_neighbors=5)
    elif(clas=='SVM RBF'):
        classifier = svm.SVC(kernel='rbf')
    """Computes for the fitness function per particle

    Inputs
    ------
    m : numpy.ndarray
        Binary mask that can be obtained from BinaryPSO, will
        be used to mask features.

    Returns
    -------
    numpy.ndarray
        Computed finess function
    """
    
    # Get the subset of the features from the binary mask
    if np.count_nonzero(m) == 0:
        X_subset = X_train
        X_subset_test= X_test
    else:
        X_subset = X_train[:,m==1]
        X_subset_test= X_test[:,m==1]
        
    
    # Perform classification and compute the fitness function
    classifier.fit(X_subset, y_train)
    j = (classifier.predict(X_subset_test) != y_test).mean() + np.count_nonzero(m)
#     j=np.count_nonzero(m)

    return j

Define fitness function for whole swarm

In [13]:
def f(x):
    """Higher-level method to do classification in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [f_per_particle(x[i]) for i in range(n_particles)]
    
    return np.array(j)

select the features from the set of features X using BPSO on the specfied parameters

In [14]:
def select_features(nparticles):
  
    total_features = X.shape[1]
    
    dimensions = total_features
    
    optimizer = ps.discrete.BinaryPSO(n_particles=nparticles, dimensions=dimensions, options=options)
    
    # Perform optimization
    cost, pos = optimizer.optimize(f, iters=iters)
    
    
    print("cost: "+str(cost))
    fr.write(clas+"-"+ds+" dataset:\ncost= "+str(cost)+"\nNumber of selected features:"+str(len([i for i in pos if i==1]))+"\nSelected features:"+str(pos))
    return cost, pos

Train clssifier on the selected features (corrposining to pos which =1)

In [15]:
def train_and_test(clas,X_train,X_test,y_train,y_test,pos, fr):   
    
    if(clas=='NB'):
        classifier = GaussianNB()
    elif(clas=='SVM linear'):
        classifier = svm.SVC(kernel='linear')
    elif(clas=='KNN'):
        classifier = KNeighborsClassifier(n_neighbors=5)
    elif(clas=='SVM RBF'):
        classifier = svm.SVC(kernel='rbf')
        
    # Get the selected features from the final positions
    X_selected_features = X_train[:,pos==1]  # subset
   
    # Perform classification and store performance in P
    classifier.fit(X_selected_features,y_train)
    selected_test= X_test[:,pos==1]
    
    y_pred= classifier.predict(selected_test)    
    report=classification_report(y_test,y_pred)
    subset_performance = (y_pred== y_test).mean()

    # Compute performance
#     subset_performance = (classifier.predict(selected_test) == y_test).mean()
    fr.write("\nsubset_performance:"+str(subset_performance)+"\n")
    fr.write("\nclassification_report:"+str(report)+"\n")
    
    print('Subset performance: %.3f' % (subset_performance))

## The main cell 

here the classifier is created, options, number of iterations are set, and other methods are called

each results for each dataset are stored in results folder with file of the classifier name

In [16]:
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
clas='NB'

# from sklearn import svm
# classifier = svm.SVC(kernel='linear')
# clas='SVM linear'

# from sklearn import svm
# classifier = svm.SVC(kernel='rbf')
# clas='SVM RBF'

# from sklearn.neighbors import KNeighborsClassifier
# classifier = KNeighborsClassifier(n_neighbors=5)
# clas='KNN'

# from sklearn import linear_model
# classifier = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto')
# clas='Linear Regression'

options = {'c1': 0.6, 'c2': 0.3, 'w': 0.4, 'k':20, 'p':2}
# k : number of neighbors to be considered. Must be a positive integer less than n_partcles 
# p: the Minkowski p-norm to use. 1 is the sum-of-absolute values (or L1 distance) while 2 is the Euclidean (or L2) distance
iters= 100


root_folder="D:\\SDP\\Afnan"
files = [os.path.join(root_folder, x) for x in os.listdir(root_folder)]
for file in files :
    start= time.time()
    
    ds=file.split("\\")[3][0:-5]
    fr = open("Test/BPSO DS2/"+clas+"/{}.txt".format(ds), "a")
    fr.write('parameters: '+str(options)+"\n")
    
    X,y = data_prepration(file)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    cost, pos = select_features(20)
    train_and_test(clas,X_train,X_test,y_train,y_test,pos,fr)
    period= time.time() - start 
    fr.write('number of particle {}:\n'.format(20))
    fr.write("best pos cost= "+str(cost)+"\nNumber of selected features:"+str(len([i for i in pos if i==1]))+"\nSelected features:"+str(pos))
    
    fr.write('\nFS, training, and testing time= '+str(period)+" seconds\n")
    fr.close()

2021-12-01 20:41:20,932 - pyswarms.discrete.binary - INFO - Optimize for 100 iters with {'c1': 0.6, 'c2': 0.3, 'w': 0.4, 'k': 20, 'p': 2}


pyswarms.discrete.binary:   0%|                                                                                  |0/100

pyswarms.discrete.binary:   0%|                                                                  |0/100, best_cost=8.19

pyswarms.discrete.binary:   0%|                                                                  |0/100, best_cost=5.15

pyswarms.discrete.binary:   0%|                                                                  |0/100, best_cost=5.15

pyswarms.discrete.binary:   0%|                                                                  |0/100, best_cost=5.15

pyswarms.discrete.binary:   0%|                                                                  |0/100, best_cost=5.15

pyswarms.discrete.binary:   0%|                                                                  |0/100, best_cost=5.15

pyswarms.disc

pyswarms.discrete.binary:  49%|███████████████████████████████▊                                 |49/100, best_cost=5.15

pyswarms.discrete.binary:  56%|████████████████████████████████████▍                            |56/100, best_cost=5.15

pyswarms.discrete.binary:  56%|████████████████████████████████████▍                            |56/100, best_cost=5.15

pyswarms.discrete.binary:  56%|████████████████████████████████████▍                            |56/100, best_cost=5.15

pyswarms.discrete.binary:  56%|████████████████████████████████████▍                            |56/100, best_cost=5.15

pyswarms.discrete.binary:  56%|████████████████████████████████████▍                            |56/100, best_cost=5.15

pyswarms.discrete.binary:  56%|████████████████████████████████████▍                            |56/100, best_cost=5.15

pyswarms.discrete.binary:  56%|████████████████████████████████████▍                            |56/100, best_cost=5.15

pyswarms.discrete.binary:  56%|█

cost: 5.154761904761905
Subset performance: 0.845


pyswarms.discrete.binary:   7%|████▌                                                             |7/100, best_cost=7.14

pyswarms.discrete.binary:   7%|████▌                                                             |7/100, best_cost=7.14

pyswarms.discrete.binary:   7%|████▌                                                             |7/100, best_cost=7.14

pyswarms.discrete.binary:   7%|████▌                                                             |7/100, best_cost=7.14

pyswarms.discrete.binary:   7%|████▌                                                             |7/100, best_cost=7.14

pyswarms.discrete.binary:   7%|████▌                                                             |7/100, best_cost=7.14

pyswarms.discrete.binary:  14%|█████████                                                        |14/100, best_cost=7.14

pyswarms.discrete.binary:  14%|█████████                                                        |14/100, best_cost=7.14

pyswarms.discrete.binary:  14%|█

pyswarms.discrete.binary:  62%|████████████████████████████████████████▎                        |62/100, best_cost=4.11

pyswarms.discrete.binary:  62%|████████████████████████████████████████▎                        |62/100, best_cost=4.11

pyswarms.discrete.binary:  62%|████████████████████████████████████████▎                        |62/100, best_cost=4.11

pyswarms.discrete.binary:  62%|████████████████████████████████████████▎                        |62/100, best_cost=4.11

pyswarms.discrete.binary:  62%|████████████████████████████████████████▎                        |62/100, best_cost=4.11

pyswarms.discrete.binary:  69%|████████████████████████████████████████████▊                    |69/100, best_cost=4.11

pyswarms.discrete.binary:  69%|████████████████████████████████████████████▊                    |69/100, best_cost=4.11

pyswarms.discrete.binary:  69%|████████████████████████████████████████████▊                    |69/100, best_cost=4.11

pyswarms.discrete.binary:  69%|█

...

pyswarms.discrete.binary:  60%|███████████████████████████████████████▌                          |60/100, best_cost=4.1

pyswarms.discrete.binary:  60%|███████████████████████████████████████▌                          |60/100, best_cost=4.1

pyswarms.discrete.binary:  60%|███████████████████████████████████████▌                          |60/100, best_cost=4.1

pyswarms.discrete.binary:  60%|███████████████████████████████████████▌                          |60/100, best_cost=4.1

pyswarms.discrete.binary:  60%|███████████████████████████████████████▌                          |60/100, best_cost=4.1

pyswarms.discrete.binary:  66%|███████████████████████████████████████████▌                      |66/100, best_cost=4.1

pyswarms.discrete.binary:  66%|███████████████████████████████████████████▌                      |66/100, best_cost=4.1

pyswarms.discrete.binary:  66%|███████████████████████████████████████████▌                      |66/100, best_cost=4.1

pyswarms.discrete.binary:  66%|█

cost: 4.101176470588236
Subset performance: 0.899


pyswarms.discrete.binary:   0%|                                                                  |0/100, best_cost=7.28

pyswarms.discrete.binary:   5%|███▎                                                              |5/100, best_cost=7.28

pyswarms.discrete.binary:   5%|███▎                                                              |5/100, best_cost=7.28

pyswarms.discrete.binary:   5%|███▎                                                              |5/100, best_cost=6.28

pyswarms.discrete.binary:   5%|███▎                                                              |5/100, best_cost=4.28

pyswarms.discrete.binary:   5%|███▎                                                              |5/100, best_cost=4.28

pyswarms.discrete.binary:   5%|███▎                                                              |5/100, best_cost=4.28

pyswarms.discrete.binary:   5%|███▎                                                              |5/100, best_cost=4.28

pyswarms.discrete.binary:  11%|█

pyswarms.discrete.binary:  59%|██████████████████████████████████████▎                          |59/100, best_cost=4.28

pyswarms.discrete.binary:  59%|██████████████████████████████████████▎                          |59/100, best_cost=4.28

pyswarms.discrete.binary:  59%|██████████████████████████████████████▎                          |59/100, best_cost=4.28

pyswarms.discrete.binary:  59%|██████████████████████████████████████▎                          |59/100, best_cost=4.28

pyswarms.discrete.binary:  59%|██████████████████████████████████████▎                          |59/100, best_cost=4.28

pyswarms.discrete.binary:  59%|██████████████████████████████████████▎                          |59/100, best_cost=4.28

pyswarms.discrete.binary:  59%|██████████████████████████████████████▎                          |59/100, best_cost=4.28

pyswarms.discrete.binary:  65%|██████████████████████████████████████████▎                      |65/100, best_cost=4.28

pyswarms.discrete.binary:  65%|█

cost: 4.27787610619469
Subset performance: 0.722
